In [38]:
import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta
# Mở kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [39]:
#  1. Tạo bảng student
cursor.execute('''
CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
)
''')

#  2. Tạo bảng course
cursor.execute('''
CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
)
''')

#  3. Chèn dữ liệu vào bảng student (một số course_id bị thiếu)
students = [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
]
cursor.executemany('INSERT INTO student VALUES (?,?,?,?,?)', students)

#  4. Chèn dữ liệu vào bảng course
courses = [
    (12, 'Giai tich'),
    (34, 'Thong ke'),
    (26, 'Tin hoc'),
    (20, 'Xac suat')
]
cursor.executemany('INSERT INTO course VALUES (?,?)', courses)


In [40]:
#  Câu 1: Kết nối hai bảng
print("\n Cartesian Product:")
df = pd.read_sql('SELECT * FROM student, course', conn)
print(df)

print("\n INNER JOIN:")
df = pd.read_sql('SELECT * FROM student INNER JOIN course ON student.course_id = course.id', conn)
print(df)

print("\n LEFT JOIN:")
df = pd.read_sql('SELECT * FROM student LEFT JOIN course ON student.course_id = course.id', conn)
print(df)

print("\n RIGHT JOIN (dùng LEFT JOIN nghịch đảo):")
df = pd.read_sql('SELECT * FROM course LEFT JOIN student ON student.course_id = course.id', conn)
print(df)



 Cartesian Product:
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  20    Xac suat
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
3            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
4            2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
5            2       Tran Thi Lan   Kinh Te       34.0    9.2  20    Xac suat
6            2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc
7            2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
8            3       Pham Van Nam  Toan Tin        NaN    7.9  12   Giai tich
9            3       Pham Van Nam  Toan Tin        NaN    7.9  20    Xac suat
10           3       Pham Van Nam  Toan Tin        NaN    7.9  26     Tin hoc
11           3       Pham Van Nam  Toan Tin

In [20]:
# Câu 2: Cập nhật dữ liệu
valid_courses = pd.read_sql("SELECT id FROM course", conn)
valid_course_ids = valid_courses["id"].tolist()

cursor.execute("SELECT student_id FROM student WHERE course_id IS NULL")
missing_students = cursor.fetchall()

for student in missing_students:
    random_course_id = random.choice(valid_course_ids)
    cursor.execute("UPDATE student SET course_id = ? WHERE student_id = ?", (random_course_id, student[0]))

cursor.execute("DELETE FROM student WHERE course_id NOT IN (SELECT id FROM course)")
conn.commit()

#  Tính toán thống kê
df_class_stats = pd.read_sql('''
SELECT class, COUNT(*) AS total_students, AVG(score) AS avg_score
FROM student
GROUP BY class
''', conn)
print("\n Tổng số sinh viên và điểm trung bình theo lớp:\n", df_class_stats)

df_course_stats = pd.read_sql('''
SELECT course.course_name, COUNT(*) AS total_students, AVG(student.score) AS avg_score
FROM student
JOIN course ON student.course_id = course.id
GROUP BY course.course_name
''', conn)
print("\n Tổng số sinh viên và điểm trung bình theo môn học:\n", df_course_stats)

df_ranking = pd.read_sql('''
SELECT course.course_name,
       AVG(student.score) AS avg_score,
       CASE
           WHEN AVG(student.score) >= 9.0 THEN 'Xuất sắc'
           WHEN AVG(student.score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
           ELSE 'Kém'
       END AS classification
FROM student
JOIN course ON student.course_id = course.id
GROUP BY course.course_name
''', conn)
print("\n Phân loại thi đua:\n", df_ranking)




 Tổng số sinh viên và điểm trung bình theo lớp:
       class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               3   7.966667

 Tổng số sinh viên và điểm trung bình theo môn học:
   course_name  total_students  avg_score
0   Giai tich               1   6.700000
1    Thong ke               3   8.533333
2     Tin hoc               2   7.450000
3    Xac suat               2   8.000000

 Phân loại thi đua:
   course_name  avg_score classification
0   Giai tich   6.700000            Tốt
1    Thong ke   8.533333            Tốt
2     Tin hoc   7.450000            Tốt
3    Xac suat   8.000000            Tốt


In [21]:
#  Câu 3: Xếp hạng sinh viên
df = pd.read_sql('''
SELECT name, score,
       RANK() OVER (ORDER BY score DESC) AS rank_overall,
       RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank_by_class,
       RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank_by_course
FROM student
''', conn)
print("\n Xếp hạng sinh viên:\n", df)

#  Lấy top 3 sinh viên cao nhất và thấp nhất
top_3 = df.nsmallest(3, 'rank_overall')
bottom_3 = df.nlargest(3, 'rank_overall')

print("\n Top 3 sinh viên có điểm cao nhất:\n", top_3)
print("\n Top 3 sinh viên có điểm thấp nhất:\n", bottom_3)



 Xếp hạng sinh viên:
                 name  score  rank_overall  rank_by_class  rank_by_course
0       Tran Thi Lan    9.2             1              1               1
1      Bui Tien Dung    9.2             1              1               1
2        Ho Ngoc Mai    8.8             3              1               1
3       Pham Van Nam    7.9             4              2               1
4     Duong Huu Phuc    7.2             5              3               3
5     Le Thanh Huyen    7.2             5              3               2
6       Cao Thi Hanh    7.0             7              1               2
7  Nguyen Minh Hoang    6.7             8              2               1

 Top 3 sinh viên có điểm cao nhất:
             name  score  rank_overall  rank_by_class  rank_by_course
0   Tran Thi Lan    9.2             1              1               1
1  Bui Tien Dung    9.2             1              1               1
2    Ho Ngoc Mai    8.8             3              1               1

 Top 3

In [35]:
import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta

#  1. Mở lại kết nối nếu bị đóng
conn = sqlite3.connect(':memory:')  # Dùng bộ nhớ tạm thời
cursor = conn.cursor()

# 2. Tạo lại bảng student nếu mất dữ liệu
cursor.execute('''
CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
)''')

#  3. Kiểm tra nếu bảng có dữ liệu, nếu không thì chèn lại
cursor.execute("SELECT COUNT(*) FROM student")
if cursor.fetchone()[0] == 0:
    students = [
        (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
        (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
        (3, 'Pham Van Nam', 'Toan Tin', 26, 7.9),
        (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
        (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
        (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
        (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
        (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
        (9, 'Duong Huu Phuc', 'Kinh Te', 26, 7.2),
        (10, 'Cao Thi Hanh', 'May Tinh', 26, 7.0)
    ]
    cursor.executemany('INSERT INTO student VALUES (?,?,?,?,?)', students)
    conn.commit()

#  4. Kiểm tra nếu cột graduation_date đã tồn tại, nếu chưa thì thêm
try:
    cursor.execute("ALTER TABLE student ADD COLUMN graduation_date TEXT")
except sqlite3.OperationalError:
    print(" Cột 'graduation_date' đã tồn tại.")

#  5. Cập nhật ngày tốt nghiệp cho từng sinh viên
cursor.execute("SELECT student_id, score FROM student")
students = cursor.fetchall()
now = datetime.now()  # Ngày hiện tại

for student in students:
    student_id, score = student

    # Tính số năm cần tốt nghiệp (10 - điểm số)
    graduation_years = max(0, int(10 - score))
    graduation_date = now + timedelta(days=graduation_years * 365)

    # Chuyển sang dạng YYYY-MM-DD
    graduation_str = graduation_date.strftime('%Y-%m-%d')

    # Cập nhật vào database
    cursor.execute("UPDATE student SET graduation_date = ? WHERE student_id = ?", (graduation_str, student_id))

#  6. Lưu thay đổi
conn.commit()

#  7. Hiển thị dữ liệu sinh viên sau khi cập nhật ngày tốt nghiệp
df_graduation = pd.read_sql("SELECT student_id, name, score, graduation_date FROM student", conn)
print("\n Dữ liệu sinh viên sau khi cập nhật ngày tốt nghiệp:\n", df_graduation)





 Dữ liệu sinh viên sau khi cập nhật ngày tốt nghiệp:
    student_id               name  score graduation_date
0           1  Nguyen Minh Hoang    6.7      2028-03-11
1           2       Tran Thi Lan    9.2      2025-03-12
2           3       Pham Van Nam    7.9      2027-03-12
3           4     Le Thanh Huyen    7.2      2027-03-12
4           5        Vu Quoc Anh    8.0      2027-03-12
5           6     Dang Thuy Linh    5.5      2029-03-11
6           7      Bui Tien Dung    9.2      2025-03-12
7           8        Ho Ngoc Mai    8.8      2026-03-12
8           9     Duong Huu Phuc    7.2      2027-03-12
9          10       Cao Thi Hanh    7.0      2028-03-11
